<a href="https://colab.research.google.com/github/tranquil-morning/CA-deeplearning/blob/main/03_Dog_vs_Cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content/"

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition 

100% 812M/814M [00:04<00:00, 198MB/s]
100% 814M/814M [00:04<00:00, 183MB/s]


In [ ]:
!unzip -q dogs-vs-cats-redux-kernels-edition.zip -d .

In [ ]:
!unzip -q train.zip -d .

In [ ]:
os.mkdir("/content/dataset")
os.mkdir("/content/dataset/dog")
os.mkdir("/content/dataset/cat")

In [ ]:
import shutil


for fn in os.listdir("/content/train/"):
  if "cat" in fn:
    shutil.move("/content/train/" + fn, "/content/dataset/cat/" + fn)
  
  if "dog" in fn:
    shutil.move("/content/train/" + fn, "/content/dataset/dog/" + fn)
  

In [ ]:
import tensorflow as tf

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/dataset/",
    image_size = (64, 64),
    batch_size = 64,
    subset = "training",
    validation_split = 0.2,
    seed = 42)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/dataset/",
    image_size = (64, 64),
    batch_size = 64,
    subset = "validation",
    validation_split = 0.2,
    seed = 42)





Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
train_ds = train_ds.map(lambda x,y: (tf.cast(x/255.0, tf.float32), y))
val_ds = val_ds.map(lambda x,y: (tf.cast(x/255.0, tf.float32), y))

In [ ]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(32, (3, 3), padding = "same", activation = tf.keras.activations.relu, input_shape = (64, 64, 3)),
                                    tf.keras.layers.MaxPooling2D((2, 2)),
                                    tf.keras.layers.Conv2D(64, (3, 3), padding = "same", activation = tf.keras.activations.relu),
                                    tf.keras.layers.MaxPooling2D((2, 2)),
                                    tf.keras.layers.Dropout(0.2), 
                                    tf.keras.layers.Conv2D(128, (3, 3), padding = "same", activation = tf.keras.activations.relu),
                                    tf.keras.layers.MaxPooling2D((2, 2)),         
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation =  tf.keras.activations.relu),
                                    tf.keras.layers.Dense(128, activation = tf.keras.activations.relu),
                                    tf.keras.layers.Dense(64, activation = tf.keras.activations.relu),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(1, activation = tf.keras.activations.sigmoid)
                                    
]) 



model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.binary_crossentropy ,
              metrics = ["accuracy"])

model.summary()



model.fit(train_ds, validation_data = val_ds)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 16, 16, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       7